In [49]:
param = {
  "input": "template.json",        # path to neo4j json dump
  "output": "data/testinput.json"  # path for output file; use "data/<something>input.json" 
                                   # so you can easily browse <viz_url>/?p=<something>
}

In [50]:
import json


def flat_map(f, xs):
    ys = []
    for x in xs:
        ys.extend(f(x))
    return ys


In [51]:
f = open(param['input'])
data = json.load(f)
f.close()

In [52]:
def to_node(item):
  return {
      "data": {
          "id": item['identity']['low'],
          "labels": item['labels'],
          "properties": item['properties']
      }
  }

def to_edge(item):
    return {
        "data": {
            "id": item['identity']['low'],
            "source": item['start']['low'],
            "target": item['end']['low'],
            "label": item['type'],
            "properties": item['properties']
        }
    }

In [53]:
nodes = list()
edges = list()
for item in sum(list(map(lambda x: x['_fields'], data)), []):
  item_nodes = [item['start'], item['end']] + sum([[i['start'],i['end']] for i in item['segments']], [])
  for item_node in item_nodes:
    node = to_node(item_node)
    if not any(n['data']['id'] == node['data']['id'] for n in nodes):
      nodes.append(node)
  item_edges = [i['relationship'] for i in item['segments']]
  for item_edge in item_edges:
    edge = to_edge(item_edge)
    if not any(e['data']['id'] == edge['data']['id'] for e in edges):
      edges.append(edge)
all = {
  "elements": {
    "nodes": nodes,
    "edges": edges
  }
}

with open(param['output'], 'w') as f:
  json.dump(all, f)